In [1]:
#Import functions to use during program
import numpy as np
import math
import matplotlib.pyplot as plt
import random as rand
import ode
import ipywidgets as widgets
import progressbar

from IPython.display import display
from vpython import *
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import Axes3D
from time import sleep
from PIL import Image

<IPython.core.display.Javascript object>

In [2]:
omaghor = np.sqrt(oldData[7]**2+oldData[9]**2)
            if(omaghor!=0):
                ohathor = ([oldData[7]/omaghor, 0, oldData[9]/omaghor])
            else:
                ohathor = ([0, 0, 0])
            
            Fg = -9.8*m
            
            omX = oldData[7] - ((2*COF*Fg)/(m*r)*0.01)*ohathor[0]
            #if(omX*oldData[5])>1:
            #    data[5] = -data[5]
            #    omX = -omX
            omZ = oldData[9] - ((2*COF*Fg)/(m*r)*0.01)*ohathor[2]
            #if(omZ*oldData[3])<1:
            #    data[3] = -data[3]
            #    omZ = -omZ
            
            data[1] = -data[1]
            data[3] = data[3] + I*r*(oldData[7]-omX)
            data[4] = -data[4]*COR
            data[5] = data[5] + I*r*(oldData[9]-omZ)
            data[7] = omX
            data[9] = omZ
            
            #print(oldData[7:], data[7:])

IndentationError: unexpected indent (<ipython-input-2-5e48d44c9bac>, line 2)

In [22]:
#Function definitions
def mag(v):           #Calculate magnitude of an array
    return np.sqrt(np.dot(v,v))

def hat(v):           #Calculate unit vector of an array
    return v/mag(v)

def cross(v1,v2):     #Calculate cross product of two arrays
    return np.cross(v1,v2)

def getCd(v):         #Return the value of Cd depending on the speed of the ball
    a = 0.36
    b = 0.14
    c = 0.27
    vc = 34
    chi = (v - vc)/4
    if chi < 0:
        Cd = a + b/(1+np.exp(chi)) - c*np.exp(-chi**2)
    else:
        Cd = a + b/(1+np.exp(chi)) - c*np.exp(-chi**2/4)
    
    return Cd

def model_magnus_2(d, tn): #Return array of derivatives
                           #rate[3]-rate[5] give accelerations due to magnus, drag, and gravity
        
    #data
    x = d[0]
    y = d[1]
    z = d[2]
    vx = d[3]
    vy = d[4]
    vz = d[5]
    omegamag = d[6]
    omega = d[7:]
    
    #derivatives
    rate = np.zeros(len(d)) #derivatives
    rate[0] = vx
    rate[1] = vy
    rate[2] = vz
    #print(x, y, z, vx, vy, vz)
    
    #speed
    v = np.array([vx,vy,vz])
    vmag = mag(v)
    
    #calculate magnus force
    Cd=getCd(vmag)
    b2 = 1/2*Cd*rho*A
    
    S = r*omegamag/vmag
    CL = 0.62*S**0.7
    alpha = 1/2*CL*rho*A*r/S
    
    FM = alpha*cross(omega,v) #magnus force
    
    speed = np.sqrt(rate[0]**2+rate[1]**2+rate[2]**2)
    
    #calculate dv/dt due to all forces
    #          [Linear Drag]  [Magnus]  [velocity dependent drag]                         [grav]
    rate[3] = -b2*vmag*vx/m + FM[0]/m - (6*np.pi*mu*r*speed*vx+0.5*Cd*rho*A*speed*vx)/m
    rate[4] = -b2*vmag*vy/m + FM[1]/m - (6*np.pi*mu*r*speed*vy+0.5*Cd*rho*A*speed*vy)/m - g
    rate[5] = -b2*vmag*vz/m + FM[2]/m - (6*np.pi*mu*r*speed*vz+0.5*Cd*rho*A*speed*vz)/m
    
    #return derivative values
    return rate

def run_magnus_2(data): # run simulation
    global b2, i, bounceDamp, gol1Pos, I, r, m, COR, mu_s, mu_k, h # need to change the value of b2 and alpha
    #Include global alpha?
    v = data[3:6]
    vmag = mag(v)

    #time
    t = 0
    h = 0.001
    Nsteps = int(10/h)

    #store trajectory for plotting or animation
    traj = []
    traj.append(np.array([t, data[0], data[1], data[2]])) #store initial time and position
    oldData = data
    #for n in range(0,Nsteps-1):
    n = 0
    while (abs(oldData[2])<=150/3.281 and abs(oldData[0])<=112.5/3.281) and n<Nsteps:

        #update data
        n+=1
        oldData = data
        data = ode.RK4(model_magnus_2, data, t, h)
        
        #Bounce off ground and lose energy
        if(data[1]<0 and n>10):
            vxz = np.sqrt(oldData[3]**2+oldData[5]**2)   #Magnitude of velocity in x,z plane
            delta_t = 0.01                               #Time ball is in contact with ground,
                                                         #may update later for further accuracy
            
            FN = m*g - 2*m*oldData[4]/delta_t            #Calculate average normal force over time of collision
            
            Ffric = mu_s*FN
            
            vxrel = oldData[3]-oldData[9]*r
            vzrel = oldData[5]+oldData[7]*r
            vrel = np.sqrt(vxrel**2+vzrel**2)
            
            F = [Ffric*vxrel/vrel, 0, Ffric*vzrel/vrel]
            Torque = [F[2]*r, 0, F[0]*r]
            delta_om = [Torque[0]*delta_t/I, 0, Torque[2]*delta_t/I]
            
            #data[3] = np.sqrt((COR*I*oldData[7]**2)/m - (I*(oldData[7]+delta_om[0])**2)/m + COR*oldData[3]**2)
            #data[4] = np.sqrt(COR*oldData[4]**2)
            #data[5] = np.sqrt((COR*I*oldData[9]**2)/m - (I*(oldData[9]+delta_om[2])**2)/m + COR*oldData[5]**2)
            vx2 = oldData[3]**2 + I/m*(np.sign(oldData[9])*oldData[9]**2-(oldData[9]+delta_om[2])**2)
            vz2 = oldData[5]**2 + I/m*(np.sign(oldData[7])*oldData[7]**2-(oldData[7]+delta_om[0])**2)
            
            data[3] = -COR*np.sign(oldData[3])*np.sqrt(abs(vx2))
            data[4] = COR*np.sqrt(oldData[4]**2)
            data[5] = COR*np.sign(oldData[5])*np.sqrt(abs(vz2))
            
            data[7] = COR*oldData[7] + delta_om[0]
            data[8] = COR*data[8]
            data[9] = COR*oldData[9] + delta_om[2]
            
            data[1] = -data[1]
            
        #update t
        t = t + h

        #store data for plotting
        #if(oldData[2]<=gol1Pos[2]):
            #print(oldData[2], gol1Pos[2], data[2])
        traj.append(np.array([t, data[0], data[1], data[2]]))
    #print(len(traj))
    return traj

In [4]:
#An innacurate but functional model with bouncing
def run_magnus_2(data): # run simulation
    global b2, alpha, i, COR, gol1Pos, h # need to change the value of b2 and alpha
    
    v = data[3:6]
    vmag = mag(v)

    #time
    t = 0
    h= 0.01
    Nsteps = int(10/h)

    #store trajectory for plotting or animation
    traj = []
    traj.append(np.array([t, data[0], data[1], data[2]])) #store initial time and position
    oldData = data
    #for n in range(0,Nsteps-1):
    n=0
    while oldData[2]<=gol1Pos[2] and n<=Nsteps:

        #update data
        oldData = data
        data = ode.RK4(model_magnus_2, data, t, h)
        
        #Bounce off ground and lose energy
        if(data[1]<0):
            data[1] = -data[1]
            data[4] = -data[4]*COR

        #update t and n
        t = t + h
        n = n + 1

        #store data for plotting
        #if(oldData[2]<=gol1Pos[2]):
            #print(oldData[2], gol1Pos[2], data[2])
        traj.append(np.array([t, data[0], data[1], data[2]]))
    return traj

In [23]:
#Will not drop out until all answers are valid
def userInput():
    userArray=np.zeros(9, dtype=float)
    while(1):
        print("What spin would you like to see?")
        print("[1] - Top spin")
        print("[2] - Side spin")
        print("[3] - Spin in direction of motion")
        print("[4] - All spin")
        UserInput = input()
        userArray[0] = int(UserInput)-1
        if(userArray[0]!=0 and userArray[0]!=1 and userArray[0]!=2 and userArray[0]!=3):
            print("That is not a valid input, please enter 1, 2, 3, or 4.")
            print("")
        else:
            break

    print("Magnitude of ball's spin in rotations per minute (600rpm is a good approximate value)")
    UserInput = input()
    userArray[1] = int(UserInput)
    if(userArray[1]==0):
        userArray[1] = 1e-10
    userArray[1] = userArray[1] * np.pi/30

    while(1):    
        print("Magnitude of ball's velocity immediately after kick in miles per hour (70mph is a good approximate value)")
        UserInput = input()
        userArray[2] = float(UserInput)
        if (userArray[2]<=0):
            print("Velocity must be greater than 0mph")
        else:
            userArray[2] = userArray[2]/2.237
            break
    while(1):
        print("How much loft would you like to give the ball? (Loft is the vertical angle relative to the ground where 0=flat along the ground and 90=directly vertical.) A standard shot at 70mph will hit the cross bar at an angle of ~17-18 degrees")
        print("WARNING: A higher loft angle will take longer to simulate due to the iteration setup")
        UserInput = input()
        userArray[3] = float(UserInput)
        if (userArray[3]<0 or userArray[3]>=90):
            print("Angle must be between 0 and 90 degrees to aim at the goal")
        else:
            userArray[3] = userArray[3] * np.pi/180
            break

    while(1):
        print("How much range would you like to give the ball? (Range is the horizontal angle relative to the center of the goal where 0=center of the goal, -90=directly to your left, and 90=directly to your right.) With no spin, an angle of +/-18.4 degress will hit the goal post")
        UserInput = input()
        userArray[4] = float(UserInput)
        if (userArray[4]<-180 or userArray[4]>=180):
            print("Angle must be between -180 and 180 degrees (0 for center of goal)")
        else:
            userArray[4] = -userArray[4] * np.pi/180
            break

    while(1):
        print("How many balls would you like to simulate? (Note: More shots will take longer to run. 100 shots takes ~25 sec to run)")
        UserInput = input()
        userArray[5] = float(UserInput)
        if(userArray[5]<=0 or userArray[5]%1!=0):
            print("Please enter a positive integer number of shots to simulate")
        else:
            userArray[5]=int(userArray[5])
            break
            
    while(1):
        print("Where would you like to shoot the ball from? (Please enter an integer value)")
        print("1.) Penalty Spot")
        print("2.) Center Spot")
        print("3.) Left Corner")
        print("4.) Right Corner")
        print("5.) Far Goal Line")
        print("6.) Custom position")
        UserInput = input()
        userArray[6] = float(UserInput)
        temp = userArray[6]
        if(userArray[6]%1!=0):
            print("Please enter an integer value between 1 and 6")
        elif(userArray[6]==1):
            userArray[6:] = [0, 0, 133/3.281]
            break
        elif(userArray[6]==2):
            userArray[6:] = [0, 0, 0]
            break
        elif(userArray[6]==3):
            userArray[6:] = [(3/4)*180/3.281, 0, 179.9/3.281]
            break
        elif(userArray[6]==4):
            userArray[6:] = [(-3/4)*180/3.281, 0, 179.9/3.281]
            break
        elif(userArray[6]==5):
            userArray[6:] = [0, 0, -180/3.281]
            break
        elif(userArray[6]==6):
            Fail=True
            while(Fail==True):
                print(Fail)
                li=[]
                string = str(input("Please enter your custom x, y, and z coordinates in feet seperated by commas (0,0,0 is at center spot)\nNote:x - across width of field\n     y - height above ground\n     z - distance down field from center spot.\n"))
                List = string.split(",")
                for i in List:
                    li.append(float(i))
                if(len(li)!=3):
                    print("Please enter 3 numerical values seperated by commas")
                else:
                    Fail=False
                    if(abs(li[0])>112.5):
                        print("Your x value is outside of the field, please enter a value between -112.5 and 112.5")
                        Fail=True
                    if(li[1]<0):
                        print("Your y value is below the field, please enter a value greater than or equal to 0")
                        Fail=True
                    if(abs(li[2])>150):
                        print("Your Z value is outside of the field, please enter a value between -150 and 150")
                        Fail=True
                    if(Fail==False):
                        userArray[6] = -li[0]/3.281
                        userArray[7] = li[1]/3.281
                        userArray[8] = li[2]/3.281
            break
            
    ###### Easter Egg!!!
    if(userArray[0]==2 and userArray[1]==np.pi/30 and userArray[2]==4/2.237 and userArray[3]==1*(np.pi/180) and userArray[4]==-5*(np.pi/180) and userArray[5]==9 and temp==2):
        print("                           .-.")
        print("                          |_:_|")
        print("                         /(_Y_)\ ")
        print("    .                   ( \/M\/ )")
        print("     '.               _.'-/'-'\-'._")
        print("       ':           _/.--'[[[[]'--.\_")
        print("         ':        /_'  : |::\"| :  '.\ ")
        print("           ':     //   ./ |oUU| \.'  :\ ")
        print("             ':  _:'..' \_|___|_/ :   :|")
        print("               ':.  .'  |_[___]_|  :.':\ ")
        print("                [::\ |  :  | |  :   ; : \ ")
        print("                 '-'   \/'.| |.' \  .;.' |")
        print("                 |\_    \  '-'   :       |")
        print("                 |  \    \ .:    :   |   |")
        print("                 |   \    | '.   :    \  |")
        print("                 /       \   :. .;       |")
        print("                /     |   |  :__/     :  \\")
        print("               |  |   |    \:   | \   |   ||")
        print("              /    \  : :  |:   /  |__|   /|")
        print("              |     : : :_/_|  /'._\  '--|_\ ")
        print("              /___.-/_|-'   \  \ ")
        print("                  '-'")
        print("        COME TO THE DARK SIDE, WE HAVE EASTER EGGS")
        print("")

    #Correct horizontal angle to aim at center of goal
    global gol1Pos
    userArray[4] = userArray[4] + math.atan((gol1Pos[0]-userArray[6])/(gol1Pos[2]-userArray[8]))
        
    return(userArray)
#userArray[0] = Case: What type of spin will be depicted
#userArray[1] = Spin Rate: Magnitude of the balls spin in RPM
#userArray[2] = Velocity: Velocity of the ball at kick in mps
#userArray[3] = Loft: The vertical angle the ball is kicked at in radians (theta)
#userArray[4] = Range: The horizontal angle the ball is kicked at in radians (phi)
#userArray[5] = N Balls: How many balls should be simulated
#userArray[6] = Color: What form of coloring should be used

In [24]:
def simulation():
    global gol1Pos
    simParam = userInput()
    
    print("Running simulation, please wait...")
    sleep(0.5)
    #print(simParam)
    modelSpins = np.zeros(3, dtype=bool)      #Select axis in which to introduce spin
    for i in range(3):
        if(simParam[0]==i or simParam[0]==3):
            modelSpins[i]=1
    #print(modelSpins)
            
    #Kick parameters such as randomized spin and inital position
    x0 = simParam[6]
    y0 = simParam[7]
    z0 = simParam[8]
    vx0 = simParam[2]*np.cos(simParam[3])*np.sin(simParam[4])
    vy0 = simParam[2]*np.sin(simParam[3])
    vz0 = simParam[2]*np.cos(simParam[3])*np.cos(simParam[4])
    
    d = ([x0, y0, z0, vx0, vy0, vz0, simParam[1], 0, 0, 0])
    #print(d)
    
    omegaHat = []
    traj = []
    bar = progressbar.ProgressBar(maxval=int(simParam[5]), \
        widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
    bar.start()
    
    for i in range(0,int(simParam[5])):
        #if(j==0):
        #    print("Ball #", j)
        #else:
        #    print("\rBall #", j)
        #Create random spins for each simulated ball
        omX = rand.randint(-100,100)
        omY = rand.randint(-100,100)
        omZ = rand.randint(-100,100)
        #omX = 50
        #omY = 50
        #omZ = 50
        #if(i==0):
        #    omX = -omX
        #    omY = -omY
        #    omZ = -omZ
        omMag = np.sqrt(omX**2+omY**2+omZ**2)
        omHat = [omX/omMag*modelSpins[0], omY/omMag*modelSpins[1], omZ/omMag*modelSpins[2]]

        omX = omX/omMag*simParam[1] * modelSpins[0]
        omY = omY/omMag*simParam[1] * modelSpins[1]
        omZ = omZ/omMag*simParam[1] * modelSpins[2]

        omegaHat.append(omHat)
        d[7:] = ([omX, omY, omZ])
        #print(d[7:], i)
        
        #Magnus Effect Calculations
        temp = run_magnus_2(d)         #Simulate model for one ball with i-th [omega] vector
        traj.append(temp)
        
        bar.update(i+1)
        
    #bar.finish()
    return traj, omegaHat

In [25]:
#Field parameters such as goal size, field length, etc
L = 36                          #ft, Distance from penalty spot to goal in ft
crossbar = 8 #ft                #Height of goal cross bar in ft
field = 360  #ft                #Length of field in ft

#Ball parameters such as diameter and mass
r = 0.11                        #.22 m diameter
r_in = 0.9*r
A = np.pi*r**2                  #cross-sectional area
m = 0.4536                      #kg
COR = 0.7                       #Coefficient of restitution
mu_s = 0.68                     #Coefficient of static friction
mu_k = 0.35                     #Coefficient of kinetic friction
I = 2/5*m*((r**5-r_in**5)/(r**3-r_in**3))                #Moment of inertia

#Physical constants such as gravity
g = 9.8                         #N/kg
rho = 1.2                       #kg/m^3
mu = 1.8e-5                     #kg/m/s

#Conversion of necessary parameters from imperial to metric
L = L/3.281                     #Convert distance to goal line to meters
crossbar = crossbar/3.281       #Convert crossbar height to meters
field = field/3.281             #Convert field length to meters
bounceDamp = 0.9                #Decimal value between 0 and 1 that represents energy conserved after
                                #ball bounces off ground

#Define positions of shot and locations of goals
shotPos = [0, 0, 133]
gol1Pos = [0, crossbar/2, 180]
gol2Pos = [0, crossbar/2, -180]
for i in range(3):              #Convert shot position and goal positions to meters
    shotPos[i] = shotPos[i]/3.281
    gol1Pos[i] = gol1Pos[i]/3.281
    gol2Pos[i] = gol2Pos[i]/3.281
diff = np.sqrt((shotPos[0]-gol1Pos[0])**2+(shotPos[1]-gol1Pos[1])**2+(shotPos[2]-gol1Pos[2])**2)
camPos = vec((shotPos[0]+gol1Pos[0])/2, (shotPos[1]+gol1Pos[1])/2, (shotPos[2]+gol1Pos[2])/2)

#Loop parameters
Nsteps = 1000                   #Number of steps each ball will take (1000 is good baseline)
t = 0                           #time
h = 0.01                        #dt

In [26]:
trajectories, spins = simulation()
iterations = 0
for i in range(len(trajectories)):
    if(len(trajectories[i])>iterations):
        iterations=len(trajectories[i])
#if iterations>100:
#    iterations=100
for i in range(len(trajectories)):
    while(len(trajectories[i])<iterations):
        trajectories[i].append(trajectories[i][-1])
print(iterations)
scene = canvas(background=vec(0.2, 0.2, 0.9))

#print(trajectories[0][3])

balls=[]
for i in range(len(trajectories)):
    if(sum(x!=0 for x in spins[0])==1):
        balls.append(sphere(radius = r, pos = vec(trajectories[0][0][1], trajectories[0][0][2]+r, trajectories[0][0][3]), color=vec(1-abs(spins[i][1])-abs(spins[i][2]), 1-abs(spins[i][0])-abs(spins[i][2]), 1-abs(spins[i][0])-abs(spins[i][1])), make_trail=True))
    else:
        balls.append(sphere(radius = r, pos = vec(trajectories[0][0][1], trajectories[0][0][2]+r, trajectories[0][0][3]), color=vec(abs(spins[i][0]), abs(spins[i][1]), abs(spins[i][2])), make_trail=True))
goal1 = box(size=vec(3*crossbar, crossbar, 0.1), pos=vec(gol1Pos[0], crossbar/2, gol1Pos[2]))
goal2 = box(size=vec(3*crossbar, crossbar, 0.1), pos=vec(gol2Pos[0], crossbar/2, gol2Pos[2]))
Field = box(size=vec(3*field/4, crossbar/8, field), pos=vec(0, -crossbar/16, 0), texture='soccerpitch.jpg')
ground = cylinder(pos=vec(0,-field-1,0), axis=vec(0,field,0), radius=field*10)

camdiff = [goal1.pos.x-balls[0].pos.x, goal1.pos.y-balls[0].pos.y, goal1.pos.z-balls[0].pos.z]
camdiff = hat(camdiff)
scene.camera.pos = balls[0].pos - 5*vec(camdiff[0], -0.5, camdiff[2])
scene.camera.axis= (balls[0].pos+goal1.pos)/2 - scene.camera.pos
scene.autoscale=False

scene.pause()

for i in range(iterations):
    rate(1/h)
    for j in range(len(balls)):
        balls[j].pos.x = trajectories[j][i][1]
        balls[j].pos.y = trajectories[j][i][2]+r
        balls[j].pos.z = trajectories[j][i][3]
        
scene.pause()

for i in range(len(balls)):
    if (balls[i].pos.z<field/2 or abs(balls[i].pos.x)>1.5*crossbar or balls[i].pos.y>crossbar):
        balls[i].clear_trail()
        balls[i].opacity = 0.2

What spin would you like to see?
[1] - Top spin
[2] - Side spin
[3] - Spin in direction of motion
[4] - All spin
3
Magnitude of ball's spin in rotations per minute (600rpm is a good approximate value)
600
Magnitude of ball's velocity immediately after kick in miles per hour (70mph is a good approximate value)
70
How much loft would you like to give the ball? (Loft is the vertical angle relative to the ground where 0=flat along the ground and 90=directly vertical.) A standard shot at 70mph will hit the cross bar at an angle of ~17-18 degrees
70
How much range would you like to give the ball? (Range is the horizontal angle relative to the center of the goal where 0=center of the goal, -90=directly to your left, and 90=directly to your right.) With no spin, an angle of +/-18.4 degress will hit the goal post
0
How many balls would you like to simulate? (Note: More shots will take longer to run. 100 shots takes ~25 sec to run)
25
Where would you like to shoot the ball from? (Please enter an

10001


<IPython.core.display.Javascript object>

TypeError: must be real number, not NoneType

## 